### **DATA VALIDATION**

In [0]:
final_df = spark.read.format("delta").load("/Volumes/olist/silver/final_df")

In [0]:
display(final_df.columns)

In [0]:
print("Total orders:", orders_df.count())
print("Total items:", order_items_df.count())
print("Total After Join orders with items:", orders_items.count())

print("Total products:", products_df.count())
print("Total After Join products with orders_items:", orders_items_products.count())

print("Total seller:", sellers_df.count())
print("Total After Join seller with orders_items_products:", orders_items_products_sellers.count())

print("Total payments:", order_payments_df.count())
print("Total After Join payments with orders_items_products_sellers:", orders_items_products_sellers_payments.count())
print("Nulls in payment_value:", orders_items_products_sellers_payments.filter("payment_value IS NULL").count())

print("Total customers:", customers_df.count())
print("Total After Join customers with orders_items_products_sellers_payments:", orders_items_products_sellers_payments_customers.count())

print("Total reviews:", order_reviews_df.count())
print("Total After Join reviews with orders_items_products_sellers_payments_customers:", orders_items_products_sellers_payments_customers_reviews.count())

print("Total geolocation:", geolocation_df.count())
print("Total After Join geolocation with orders_items_products_sellers_payments_customers_reviews:", orders_items_products_sellers_payments_customers_reviews_geolocation.count())

print("Total names:", products_name_translation_df.count())
print("Total After Join names with orders_items_products_sellers_payments_customers_reviews_names:", final_df.count())

In [0]:
print([col for col in final_df.columns if final_df.columns.count(col) > 1])

In [0]:
display(final_df.select([count(when(col(c).isNull(), c)).alias(c) for c in final_df.columns]))

In [0]:
from pyspark.sql import functions as F

display(order_payments_df.select(*[
    (
        F.count(F.when((F.isnan(c) | F.col(c).isNull()), c)) if t in ("int", "double", "float")
        else F.count(F.when(F.col(c).isNull(), c))
    ).alias(c)
    for c, t in order_payments_df.dtypes if c in order_payments_df.columns
]))

In [0]:
from pyspark.sql.functions import col

def filter_invalid_hex_ids(df, column_name, show_invalid=True):
    """Filter out rows with invalid hexadecimal IDs in the specified column."""
    pattern = r"^[a-fA-F0-9]{32}$"

    # Identify invalid rows
    invalid_rows_df = df.filter(~col(column_name).rlike(pattern))
    invalid_count = invalid_rows_df.count()

    if show_invalid:
        if invalid_count > 0:
            print(f"❌ Found {invalid_count} invalid rows in column '{column_name}':")
            display(invalid_rows_df)
        else:
            print(f"✅ No invalid rows found in column '{column_name}'.")

    # Return only valid rows
    return df.filter(col(column_name).rlike(pattern))

In [0]:
filter_invalid_hex_ids = filter_invalid_hex_ids(final_df, 'review_order_id')

In [0]:
def uniqueness(df_name, df_name_str, column_names):
    print(f'Dataframe is {df_name_str}')
    for column_name in column_names:
        print(f"Checking uniqueness for column: {column_name}")
        display(df_name.groupBy(column_name).count().filter("count > 1"))

In [0]:
# uniqueness(customers_df, 'customers_df', ['customer_id', 'customer_unique_id'])
# uniqueness(orders_df, 'orders_df', ['order_id', 'customer_id'])
# uniqueness(order_items_df, 'order_items_df', ['order_id', 'order_item_id', 'product_id', 'seller_id'])
# uniqueness(order_payments_df, 'order_payments_df', ['order_id'])
# uniqueness(sellers_df, 'sellers_df', ['seller_id', 'seller_zip_code_prefix'])
# uniqueness(products_df, 'products_df', ['product_id', 'product_category_name'])
# uniqueness(order_reviews_df, 'order_reviews_df', ['review_id', 'review_order_id'])
uniqueness(geolocation_df, 'geolocation_df', ['geolocation_lat'])
# uniqueness(products_name_translation_df, 'products_name_translation_df', ['product_category_name', 'product_category_name_english'])